In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings

In [20]:
class RAGPipeline:
    def __init__(self, llm_provider = "groq", llm_model=None, api_key=None, embed_model=None, data_dir="data"):
        self.llm= self._init_llm(llm_provider, llm_model, api_key)
        self.embed_model = embed_model or "local:BAAI/bge-small-en-v1.5"
        self.data_dir = data_dir
        self.index = None
        self.query_engine = None
        self._setup()

    def _init_llm(self, provider, model, api_key):
        if provider == 'groq':
            from llama_index.llms.groq import Groq
            return Groq(model=model or 'llama3-70b-8192',
                       api_key = api_key)
        elif provider == "openai":
            from llama_index.llms.openai import OpenAI
            return OpenAI(model= model or 'gpt-4o',
                         api_key = api_key)
        elif provider == 'replicate':
            from llama_index.llms.replicate import Replicate
            return Replicate(model=model or "meta/meta-llama-3-70b-instruct",
                            api_key = api_key)
        else:
            raise ValueError("Unsupported LLM provider")
    def _setup(self):
        Settings.llm = self.llm
        Settings.embed_model = self.embed_model
        documents = SimpleDirectoryReader(self.data_dir).load_data()
        self.index = VectorStoreIndex.from_documents(documents)
        self.query_engine= self.index.as_query_engine(similarity_top_k=3, 
                                                     respose_mode = "compact")

    def query(self, question):
        return self.query_engine.query(question)

In [23]:
# if __name__ == "__main__":
llm_provider = "groq"
groq_api_key = ""
groq_llm_model = "compound-beta"
embed_model = "local:BAAI/bge-small-en-v1.5"
data_dir = "data"
rag = RAGPipeline(llm_provider, groq_llm_model, api_key=groq_api_key, data_dir=data_dir)

In [24]:
rag.query("Who is vishal")

Response(response='Vishal Babu is a Lead Data Scientist with over 9.5 years of experience in building and deploying AI/ML systems at scale. He specializes in Generative AI, LLMs, RAG pipelines, and intelligent agent frameworks. Vishal has a strong background in NLP, computer vision, and multimodal models, with expertise in modern ML stacks and data engineering. He has worked on various projects, including clinical note generation, sales call analysis, and real-time threat detection. \n\nVishal holds an M.Tech in Computer Science and Engineering from the National Institute of Technology Karnataka and a B.Tech in Computer Science and Engineering from the United College of Engineering and Research. He has a strong educational foundation and has worked with various companies, including InstantSys, Blackstraw.ai, Equinix, Larsen & Toubro Infotech, and AutoRABIT Inc.\n\nAs a skilled data scientist, Vishal has developed a range of technical skills, including programming languages such as Pyth

In [27]:
REPLICATE_API_TOKEN=""
replicate_model_name = "mistralai/mistral-7b-v0.1"
llm_provider = "replicate"
embed_model = "local:BAAI/bge-small-en-v1.5"
data_dir = "data"
replicate_rag = RAGPipeline(llm_provider, llm_model=replicate_model_name, api_key=REPLICATE_API_TOKEN, data_dir=data_dir)